# Aula 06: WebScraping

Um dos maiores desafios de Ciência de Dados é conseguir os Dados. 

Uma forma é conseguir dados com o cliente/sua empresa e analisá-los.

Uma outra forma de conseguir é através de **[APIs](https://codeburst.io/6-interesting-apis-to-check-out-in-2018-5d6830063f29)**, interfaces para se conseguir dados oficialmente de fontes confiáveis. 

O **Facebook** tinha uma ótima API. Tinha. Depois do escândalo da Cambridge Analytica, boa parte das funções interessantes foi tirada.

O **[Instagram](https://www.instagram.com/developer/)**, **[Twitter](https://developer.twitter.com/en/docs.html)** e a **[NASA](https://api.nasa.gov/)** tem ótimas APIs para diversas aplicações, recomendo ir atrás delas.

Quando isso não é possível (a fonte não tem uma API, ou os dados estão **desestruturados** e em *sites*, existe o **WebScraping**!

#### Os algoritmos de WebScraping fazem um *request*, pegam o código-fonte da página e buscam a informação que você pedir nele
---

In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs

Vamos buscar informações sobre o histórico de uso de um site, através do site [Rank2Traffic](https://www.rank2traffic.com/).

In [ ]:
url = 'https://www.rank2traffic.com/stackoverflow.com' # Because Stack Overflow is God.

In [ ]:
session = req.Session() # Começando a sessão, como se estivesse abrindo o browser
# Os headers fazem o Python fingir ser um browser comum, para evitar que o site bloqueie ele
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8"}
ref = session.get(url, headers=headers) # Aqui ele abre o site e puxa o código-fonte
soup = bs(ref.text, "html.parser") # E transforma o código em texto (método .text) e logo depois em um objeto BeautifulSoup
print(soup) # que tem essa cara

In [ ]:
# Extraindo a tabela
tab = soup.find('table', {'id':"myTab1-table"}) # método .find: encontre a primeira tabela com id 'myTab1-table'
rows=list() # para salvar as linhas da tabela
for row in tab.findAll("tr"):
    try:
        rows.append(row.findAll("td")[1].string.replace(" ","").replace("\n",""))# a segunda coluna (índice 1) é a que interessa
    except IndexError: # na primeira linha, não tem índice 1. então coloque o nome do site 
        rows.append(url[-17:])

In [ ]:
# Extraindo dados complementares de uso
meta = soup.findAll('div',{'class':'infobox-data'})[1:] # método .findAll: Encontre TODOS nessas condições, e retorne uma lista
for box in meta:
    rows.append(box.find('span').string)
    rows.append(box.find('div').string)

In [ ]:
# Extraindo percentual de acessos por país
tab = soup.find('table', {'class':"table table-hover table-condensed"})
try: 
    for row in tab.findAll("tr"):
        try:
            rows.append(row.findAll("td")[0].string)
            rows.append(row.findAll("td")[1].string)
        except IndexError:
            rows.append('Paises')
except AttributeError:
    rows.append('Supondo, sem info oficial')
    rows.append('100%')

In [ ]:
rows

In [ ]:
import pandas as pd
df = pd.DataFrame(rows)
df.to_excel("StackOverflow.xlsx")